See: https://towardsdatascience.com/getting-the-most-out-of-scikit-learn-pipelines-c2afc4410f1a

In [ ]:
import gzip
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
import requests
import pandas as pd
from io import BytesIO, StringIO

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
def read_gzip_pickle_data_to_pd(url):

# This is required to access raw binary files on Github
# i.e. it appends the following to the URL: `?raw=true`.

    query = {'raw': 'true'} 

# Use the requests module to parse the URL with the provided parameters.
    response = requests.get(url, params=query)

# Create a file pointer initialized to the content of the response
# using `BytesIO`. This is a pseudo-file, which can now be read
# using `pandas.read_csv`. Since `response.content` is binary data
# i.e. bytes, we use `BytesIO`. If the response was text, we would
# have used `StringIO`.

    fp = BytesIO(response.content)

# Finally, parse the content into a DataFrame 
# (populate other parameters as needed).

    df = pd.read_pickle(fp, compression='gzip')

    return df

In [ ]:
url = "https://github.com/jess-miles/nyt-engagement/blob/master/data/cleaned_data.pickle.gz"

df1 = read_gzip_pickle_data_to_pd(url)

In [ ]:
url = "https://github.com/jess-miles/nyt-engagement/blob/master/data/cleaned_article_data.pickle.gz"

df = read_gzip_pickle_data_to_pd(url)

In [ ]:
df.head()

In [ ]:
# TODO: Perhaps need to join

In [ ]:
df.info()

In [ ]:
# TODO: Might need to join these datasets to get all columns?

In [ ]:

# sets of columns to be transformed different ways

text_col = 'article_text'
topic_col = 'topics'
cat_cols = ['section_name', 'word_count_cat', 'is_multimedia', 'on_weekend']

# initial X and y set for all posts

X = df[[text_col, topic_col] + cat_cols]
y = df['target']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
cols_trans = ColumnTransformer([
    ('txt', TfidfVectorizer(), text_col),
    ('txt_kw', CountVectorizer(), topic_col),
    ('ohe', OneHotEncoder(drop='first'), cat_cols)], remainder='drop')

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipe = Pipeline([
    ('trans', cols_trans),
    ('clf', LogisticRegression(max_iter=300, class_weight='balanced'))
])

In [ ]:
from sklearn import set_config
set_config(display='diagram')

# with display='diagram', simply use display() to see the diagram

display(pipe)

# if desired, set display back to the default
set_config(display='text')